In [59]:
import tensorflow as tf
import numpy as np
import json

In [158]:
DATA_DIR='data/shakespeare.txt'
BATCH_SIZE=25
LAYER_NUM=2
SEQ_LENGTH=50
HIDDEN_DIM=128
GENERATE_LEN=500
NB_EPOCH=1500
MODE='train'
WEIGHTS=''
LEARNING_RATE=2e-3
IS_TRAIN=True

In [159]:
def get_text_data():
    with open(DATA_DIR,"r") as text_file:
        data=text_file.read()
#         tr_data,va_data=data[0:1070392],data[1070392:1075392]
    lis=list(set(data))    
    total_char,unique_char=len(data),len(lis)
    print ('data has %d characters, %d unique.' %(total_char, unique_char))
    char_to_ix = { ch:i for i,ch in enumerate(lis) }
    ix_to_char = { i:ch for i,ch in enumerate(lis) }
    return data

def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [160]:
data=get_text_data()

data has 1115393 characters, 65 unique.


In [161]:
# set(data)

In [162]:
# Using the char-to-id mappings that i used earlier and stored in this file, used for training if stopped at some time

with open('data/coding2.json') as file:
    da=json.load(file)
    total_chars,unique_chars,char_id,id_char=da['total'],da['unique'],da['char_id'],da['id_char']

id_v={int(k):v for k,v in id_char.items()}

In [163]:
def generate_next_batch(given_data, seq_length, p):
    inputs = [char_id[ch] for ch in given_data[p:p+seq_length]]
    targets = [char_id[ch] for ch in given_data[p+1:p+seq_length+1]]
    inputs=np.array(inputs).reshape(1,seq_length)
    targets=np.array(targets).reshape(1,seq_length)
    return inputs,targets

In [164]:
#Preprocessing- generated all the batches at once rather than dynamic generation
ptr=0
batches=(total_chars//SEQ_LENGTH)
x=[]
y=[]
for batch in range(batches):
    a,b=generate_next_batch(data, SEQ_LENGTH, ptr)
    x.append(a)
    y.append(b)
    ptr+=SEQ_LENGTH
x=np.array(x).reshape(batches, SEQ_LENGTH)
y=np.array(y).reshape(batches, SEQ_LENGTH)

In [165]:
y.shape

(44615, 25)

In [166]:
#smaple generator function here, tbh this is a hack where i feed a sequence of 50 chars but just 
#use the first one and replace it, if any way to reset the x and y tensors- definitely help
def generator(seed_i,n,cu_state):
    x= seed_i
    ixes=[]
    ixes.append(seed_i[0][0])
    global IS_TRAIN
    IS_TRAIN=False
    for t in range(n):
        p,cu_state=sess.run([prediction,states],{X:x,_initial_state:cu_state})
        ix = np.random.choice(range(unique_chars), p=p[0].ravel())
        x[0][0]=ix
        ixes.append(ix)
    return ixes

In [167]:
def build_LSTM(x):
    with tf.name_scope('lstm'):
        cells= [tf.contrib.rnn.BasicLSTMCell(num_units=HIDDEN_DIM) for layer in range(LAYER_NUM)]
        multi_layer_cell = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)
        return multi_layer_cell

In [ ]:
# building the graph, again i wanted this to be as simple as andrej's code so didnt want to implement this as a class,
# but i have uploaded a different class based version in the second ipython file
reset_graph()

X=tf.placeholder(tf.int32,[BATCH_SIZE,SEQ_LENGTH],name='inputs')
Y=tf.placeholder(tf.int32,[BATCH_SIZE,SEQ_LENGTH],name='targets')

X_onehot=tf.one_hot(X,unique_chars)
Y_onehot=tf.one_hot(Y,unique_chars)

multi_layer_cell=build_LSTM(X_onehot)

_initial_state =  multi_layer_cell.zero_state(BATCH_SIZE, tf.float32)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell, X_onehot,initial_state=_initial_state,dtype=tf.float32)

with tf.name_scope('mine'):
    W=tf.Variable(tf.random_normal((HIDDEN_DIM,unique_chars),stddev=0.1))
    B=tf.Variable(tf.zeros(shape=[unique_chars]))

with tf.name_scope('logits'):
    flat_outputs = tf.reshape(tf.concat(axis=1, values=outputs), [-1, HIDDEN_DIM])
    flat_y = tf.reshape(tf.concat(axis=1, values=Y_onehot), [-1,unique_chars])
    Ys=tf.matmul(flat_outputs,W)+B

with tf.name_scope('cross-entropy'):
    prediction=tf.nn.softmax(Ys)
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=flat_y, logits=Ys)
    loss = tf.reduce_mean(xentropy)

with tf.name_scope('adam_optimizer'):
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), 5)
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
    minimize = optimizer.apply_gradients(zip(grads,tvars))

In [ ]:
saver=tf.train.Saver()
init=tf.global_variables_initializer()

no_of_batches=int(total_chars//BATCH_SIZE)
ptr=0
n=0

In [1]:
#this was running as expected but it has some errors, and even when the loss reached 3, satisfactory results were not obtained

# with tf.Session() as sess:
#     smooth_loss_tr = -np.log(1.0/unique_chars)*SEQ_LENGTH # loss at iteration 0
#     sess.run(init)
#     zero_state=sess.run(_initial_state)
#     for i in range(NB_EPOCH):
#         curr_state=zero_state
#         for j in range(batches//BATCH_SIZE):
#             if (ptr+BATCH_SIZE+1 >= batches): 
#                 ptr=0
#             inputs,targets=x[ptr:ptr+BATCH_SIZE],y[ptr:ptr+BATCH_SIZE]
#             ptr+=BATCH_SIZE
            
#             if(j%100==0): 
#                 sample_ix=generator(np.array(inputs),200,zero_state)
#                 txt = ''.join(id_v[ix] for ix in sample_ix)
#                 print ('----\n %s \n----' % (txt, ))
            
#             _,loss_t,curr_state=sess.run([minimize,loss,states],feed_dict={X:inputs,Y:targets,_initial_state:curr_state})
#             smooth_loss_tr = smooth_loss_tr * 0.999 + loss_t * 0.001
            
#             if(j%100==0):
#                 print('epoch %i iteration %i loss %f va'%(i,j,smooth_loss_tr))
        
#         if(i%10==0):
#             save_path = saver.save(sess, "finall/LSTMmodel"+str(i)+"loss"+str(int(smooth_loss_tr))+".ckpt")

In [ ]:
#here is where I found some trouble, as can be seen from the above cell i changed my implementation to fit it more like
#karpathy's min char rnn but the smooth_loss drops too rapidly

with tf.Session() as sess:
    smooth_loss= -np.log(1.0/unique_chars)*SEQ_LENGTH # loss at iteration 0
    sess.run(init)
    zero_state=sess.run(_initial_state)
    curr_state=zero_state
    while True:
        if(ptr+BATCH_SIZE+1>=y.shape[0]):
            curr_state=zero_state
            ptr=0
        inputs,targets=x[ptr:ptr+BATCH_SIZE],y[ptr:ptr+BATCH_SIZE]
        
        if n % 100 == 0:
            sample_ix = generator(inputs, 200, curr_state)
            txt = ''.join(id_v[ix] for ix in sample_ix)
            print ('----\n %s \n----' % (txt, ))
        
        loss_t,curr_state,_=sess.run([loss, states, minimize], feed_dict={X:inputs, Y:targets,
                                                                        _initial_state:curr_state})
        smooth_loss = smooth_loss * 0.999 + loss_t * 0.001

        if n % 100 == 0: print ('iter %d, loss: %f' % (n, smooth_loss)) # print progress
        ptr+=BATCH_SIZE
        n+=1